In [22]:
import numpy as np
import tensorflow.keras as keras
from IPython.display import clear_output
from keras.models import Model
from keras.layers import Dense, Input
from IPython.display import display

EMPTY = 1;
COLOR = 0;

BLACK = -1;
WHITE = 1;

WIDTH = 9;

def index_to_coordinate(index): #Returns row, column
    return int(index / WIDTH), index % WIDTH

def coordinate_to_index(x, y): #/Returns index
    return y*WIDTH+x


In [23]:
def encode(board, color): # Converts a board position into a string, to reduce the memory needed to store past position states
    space = 0
    out = str(color)
    for x in range(WIDTH): # Loop through each tile on the board and encode it
        for y in range(WIDTH):
            if board[y, x, EMPTY] == 0: # Non-Empty Tile
                if board[y, x, COLOR] == BLACK: # Black piece
                    out += "b"
                if board[y, x, COLOR] == WHITE: # White piece
                    out += "w"
                if space > 0: # Show that the previous streak of empty tiles has ended.
                    out += "." + str(space) + "."
                    space = 0
            else: # Space tells how many empty squares there were in a row before a piece was found. An empty board would be encoded as "" (best case), a full board as wbwbwbwb. . . (worst case)
                space += 1
    return out

def decode(notation): # Decodes a string created by the encode function and converts it to an array of length 81.
    index = 1
    strIndex = 0
    skip = 0
    board = np.zeros((WIDTH*WIDTH, 2))
    color = notation[0]
    board[:] = -1
    while index < WIDTH*WIDTH and strIndex < notation.size():
        coord = index_to_Coordinate(index)
        x = coord[0]
        y = coord[1]
        if notation[strIndex] == 'b' or notation[strIndex] == 'w':
            if notation[strIndex] == 'b': # Black Piece on this tile
                if color == -1:
                    board[y, x, COLOR] = 1
                else:
                    board[y, x, COLOR] = -1
            else:                         # White piece on this tile
                if color == -1:
                    board[y, x, COLOR] = -1
                else:
                    board[y, x, COLOR] = 1
            index += 1
        elif notation[strIndex] == '.':   # Empty tiles
            skip = 0
            strIndex += 1
            while notation[strIndex] != '.': # Streaks of empty tiles are coded as numbers followed by a period. This reads how many empty tiles there were, stopping once it sees a period.
                skip *= 10
                skip += notation[strIndex] - '0'
                strIndex += 1
            index += skip
    return board, color

def Decode_Move(move):
    if move == "":
        return 81
    else:
        xC = ord(move[0]) - ord('a')
        yC = ord(move[1]) - ord('a')
        return xC+yC*WIDTH

def createEmptyBoard():
    Board = np.zeros((WIDTH, WIDTH, 2))
    for x in range(WIDTH):
        for y in range(WIDTH):
            Board[y, x, EMPTY] = 1
    return Board


In [24]:
def Flood(board, y1, x1, color): # Performs a flood fill to check if tiles are completely surrounded. If they are, then it returns a 1 and a list of the pieces to be captured.
    closed = [] # Closed List
    open_list = [x1+y1*WIDTH] # Open List

    x = x1
    y = y1
    if board[y, x, COLOR] != color:
        return -1, []
    while len(open_list) > 0:
        index = open_list[-1]
        x = index % WIDTH # X coordinate
        y = int(index / WIDTH) # Y coordinate
        closed.append(open_list.pop())

        for a in range(-1, 2): # This nested loop checks in the 4 cardinal directions adjacent to a tile
            for b in range(-1, 2):
                if (a == 0 or b == 0) and a != b: # <- make sure not to check diagonals
                    if a + y >= 0 and b + x >= 0 and a + y < WIDTH and b + x < WIDTH: # Make sure coords aren't out of bounds
                        idx = (x+b)+(y+a)*WIDTH # index, for simplification
                        if board[y+a, x+b, EMPTY] == 1: # If it's empty, stop searching, there will be no capture here.
                            return -1, []
                        if board[y+a, x+b, COLOR] == color: # If it's another allied stone, check to see if its liberties are taken
                            if not idx in closed and not idx in open_list: # Make sure that stone hasn't already been checked/will be checked to avoid infinite loops
                                open_list.append(idx)
    return 1, closed # No open tiles were ever found to stop the loop, so all liberties were taken.

def flipBoard(bd):
    board = bd
    for x in range(WIDTH):
        for y in range(WIDTH):
            board[y, x, COLOR] *= -1
    return board

def Move(bd, y, x): # Make a move. Return 1, board if successful, -1, parameter board if unsuccessful (ie suicide move)
    # Make a move at the index. color 1 = white, color -1 = black
    board = np.array(bd) # This is done to make a shallow copy of the parameter

    board[y, x, COLOR] = 1 # This is where the move is made. 
    board[y, x, EMPTY] = 0

    # Check for captured pieces:

    captured = []
    if y > 0: # Check for capture 1 sq up
        captured += Flood(board, (y-1), x, -1)[1]

    if y < WIDTH-1: # Check for capture 1 sq down
        captured += Flood(board, (y+1), x, -1)[1]

    if x > 0: # Check for capture 1 sq left
        captured += Flood(board, y, (x-1), -1)[1]

    if x < WIDTH-1: # Check for capture 1 sq right
        captured += Flood(board, y, (x+1), -1)[1]

    if len(captured) > 0:
        for i in captured:
            ape = index_to_coordinate(i)
            board[ape[0], ape[1], EMPTY] = 1
            board[ape[0], ape[1], COLOR] = 0
    
    if Flood(board, y, x, 1)[0] != -1: # Check for Suicides
        return -1, bd
    board = flipBoard(board)
    return 1, board


def createMask(board, positions, color):
    mask = np.zeros(WIDTH* WIDTH)
    for x in range(WIDTH):
        for y in range(WIDTH):
            if board[y, x, EMPTY] == 1:
                mask[x+WIDTH*y] = 1
            current = ""
            for a in range(WIDTH*WIDTH):
                if mask[a]:
                    variation = Move(board, int(a/WIDTH), a%WIDTH)
                    if variation[0] == -1:
                        mask[a] = -1
                    else:
                        if len(positions) > 0:
                            position = encode(variation[1], color*-1)
                            if position in positions:
                                mask[a] = -1
    return mask

In [27]:
def printBoard(board, color):
    string = ""
    for y in range(-1, WIDTH):
        for x in range(-1, WIDTH):
            if x < 0:
                if y < 0:
                    string += '# '
                else:
                    string += str(y+1)+" "
            elif y < 0 and x > -1:
                string += ' ' + chr(65+x)
            else:
                if color == BLACK:
                    if board[y, x, COLOR] == 1:
                        string += ' @'
                    elif board[y, x, COLOR] == -1:
                        string += ' O'
                    else:
                        string += ' .'
                elif color == WHITE:
                    if board[y, x, COLOR] == 1:
                        string += ' O'
                    elif board[y, x, COLOR] == -1:
                        string += ' @'
                    else:
                        string += ' .'
        string += '\n'
    print(string)

def stringToIndex(mv): 
    # Output 81 for pass,
    # 0-80 for a1-i8
    #-1 if invalid
    move = mv.lower()
    xC = 0
    yC = 0
    index = 0
    
    if move == "pass":
        return 81
    else:
        xC = ord(move[0]) - ord('a')
        yC = ord(move[1]) - ord('1')
        if xC < 0 or xC > WIDTH or yC < 0 or yC > WIDTH:
            return -1
        return xC+yC*WIDTH

def getEngineMove(bd, msk): # Get the index which the engine wants to move to.
    mask = np.append(msk, 1)
    net_moveList = (mask * model(np.array([bd]), training=False)[0])
    net_move = 0
    for x in range(len(net_moveList)):
        if net_moveList[x] > net_moveList[net_move]:
            net_move = x
    return net_move

def Play():
    board = createEmptyBoard() # Board here is an 81 size array, but net has to have 82 size input
    print(board.shape)
    positions = []
    color = -1
    pass_count = 0
    val = ""
    xC = 0
    yC = 0

    while val != "quit" and pass_count < 2:
        positions.append(encode(board, color))
        mask = createMask(board, positions, color)
        #clear_output(wait=True) # IF THE CODE IS FREEZING UP DURING GAMEPLAY, DELETE/COMMENT THIS LINE. JUPYTER SOMETIMES FREEZED TRYING TO CLEAR OUTPUT
        printBoard(board, color)
        if color == BLACK: # The net plays black's moves here. if TRUE -> net plays both sides. if FALSE -> player plays both sides. "if color == BLACK:" net plays black, player plays white
            if color == BLACK:
                string = "Black's Move: "
            else: string = "White's Move: "
            net_move = getEngineMove(board, mask)
            if net_move < WIDTH*WIDTH:
                board = Move(board, int(net_move/WIDTH), net_move%WIDTH)[1]
                pass_count = 0
            else:
                pass_count += 1
                board = flipBoard(board)
            color *= -1
            string += ": " + str(net_move) # Print what index the net choses to play at.
            print(string)
        else:
            if color == BLACK:
                string = "Black's Move: "
            else: string = "White's Move: "
            val = input(string)
            if val == "pass":
                color *= -1
                pass_count += 1
                board = flipBoard(board)
            elif val != "quit":
                pass_count = 0
                val = stringToIndex(val)
                if val >= 0 and val < 81 and mask[val] == 1: # Check if the move is legal
                    board = Move(board, int(val/WIDTH), val%WIDTH)[1] # Make the move
                    color *= -1
                else:
                    print("Illegal move!", end = "\n")
    if pass_count >= 2:
        print("Players agreed to end the game.", end = "\n")
#Play()